In [1]:
#Installer le package de base
%pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
#package de base
from kafka import KafkaProducer
from kafka.errors import KafkaError
from kafka import KafkaConsumer
import sys

In [4]:
#Installation
%curl -sSOL https://downloads.apache.org/kafka/3.7.2/kafka_2.12-3.7.2.tgz
%tar -xzf kafka_2.12-3.7.2.tgz

UsageError: Line magic function `%curl` not found.


In [5]:
!ls -l

total 118068
-rw-r--r-- 1 root root       196 Jun 26 01:18 kafka_2.12-3.7.1.tgz
drwxr-xr-x 7 root root      4096 Dec  4  2024 kafka_2.12-3.7.2
-rw-r--r-- 1 root root 120886836 Jun 26 01:20 kafka_2.12-3.7.2.tgz
drwxr-xr-x 1 root root      4096 Jun 24 13:38 sample_data


In [6]:
#Demarrage
!./kafka_2.12-3.7.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.7.2/config/zookeeper.properties
!./kafka_2.12-3.7.2/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.7.2/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [7]:
#Verifier que c'est bien démarré
!ps -ef | grep kafka

root        1992       1 11 01:21 ?        00:00:01 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-3.7.2/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-3.7.2/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-3.7.2/bin/../config/log4j.properties -cp /content/kafka_2.12-3.7.2/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-3.7.2/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-3.7.2/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-3.7.2/bin/../libs/audience-annotations-0.12.0.jar:/content/kafka_2.12-3.7.2/bin/../libs/caffeine-2.9.3.jar:/content/kafka_2.12-3.7.2/bin/../libs/checker-qual-3.19.0.jar:/content/kafka_2.12

In [8]:
#Créer un topic
!./kafka_2.12-3.7.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic obd2
!./kafka_2.12-3.7.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic temperature


Created topic obd2.
Created topic temperature.


In [9]:
#Décrire les topics
!./kafka_2.12-3.7.2/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic obd2
!./kafka_2.12-3.7.2/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic temperature

Topic: obd2	TopicId: VHp5_9p1SEmetwIf5Busww	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: obd2	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: temperature	TopicId: xnZRr2opTdSYaj0cEcYUdw	PartitionCount: 2	ReplicationFactor: 1	Configs: 
	Topic: temperature	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: temperature	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


In [10]:
#Envoyer le data vers Kafka
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for key, message in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

obd_a = ("cam1", "cam2", "cam3")
obd_b = ("111", "121", "131")
write_to_kafka("obd2", zip(obd_a, obd_b))

temp_a = ("cam1", "cam2", "cam3","cam4", "cam5")
temp_b = ("231", "221", "24", "500", "110")
write_to_kafka("temperature", zip(temp_a, temp_b))

Wrote 3 messages into topic: obd2
Wrote 5 messages into topic: temperature


In [11]:
#Streamer le data de Kafka vers le processeur/consumer OBD2

topic = 'obd2'
consumer1 = KafkaConsumer(
    topic,
     bootstrap_servers=['127.0.0.1:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     )

try:
    for message in consumer1:
        print ("{} {} {}: key={} value={}".format(message.topic, message.partition,message.offset, message.key,message.value))
    #sys.exit()
except KeyboardInterrupt:
    sys.exit()

obd2 0 0: key=b'cam1' value=b'111'
obd2 0 1: key=b'cam2' value=b'121'
obd2 0 2: key=b'cam3' value=b'131'


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/tmp/ipython-input-11-2506760486.py", line 13, in <cell line: 0>
    for message in consumer1:
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 1188, in __next__
    return next(self._iterator)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 1160, in _message_generator_v2
    record_map = self.poll(timeout_ms=timeout_ms, update_offsets=False)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 684, in poll
    records = self._poll_once(timer, max_records, update_offsets=update_offsets)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 731, in _poll_once
    self._client.poll(timeout_ms=poll_timeout_ms)
  File "/usr/local/lib/python3.11/dist-packag

TypeError: object of type 'NoneType' has no len()

In [12]:
#Streamer le data de Kafka vers le processeur/consumer Temp

topic1 = 'temperature'
consumer2 = KafkaConsumer(
    topic1,
     bootstrap_servers=['127.0.0.1:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     )

try:
    for message in consumer2:
        print ("{} {} {}: key={} value={}".format(message.topic, message.partition,message.offset, message.key,message.value))

except KeyboardInterrupt:
    sys.exit()

temperature 0 0: key=b'cam2' value=b'221'
temperature 1 0: key=b'cam1' value=b'231'
temperature 1 1: key=b'cam3' value=b'24'
temperature 1 2: key=b'cam4' value=b'500'
temperature 1 3: key=b'cam5' value=b'110'


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/tmp/ipython-input-12-1862822242.py", line 13, in <cell line: 0>
    for message in consumer2:
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 1188, in __next__
    return next(self._iterator)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 1160, in _message_generator_v2
    record_map = self.poll(timeout_ms=timeout_ms, update_offsets=False)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 684, in poll
    records = self._poll_once(timer, max_records, update_offsets=update_offsets)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kafka/consumer/group.py", line 731, in _poll_once
    self._client.poll(timeout_ms=poll_timeout_ms)
  File "/usr/local/lib/python3.11/dist-packag

TypeError: object of type 'NoneType' has no len()